In [1]:
print('\nEnabling interactive shell outputs ...')
print('   Use command pass; to disable cell text outputs')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings('ignore') 
warnings.simplefilter(action="ignore",category=UserWarning)
warnings.simplefilter(action="ignore",category=FutureWarning)

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}


Enabling interactive shell outputs ...
   Use command pass; to disable cell text outputs


In [2]:
growth_marigold_df = pd.read_excel("data/growth_marigold_sprout.xlsx")
growth_marigold_df

,A,B,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-1,-1,34.2,26.2,32.6,33.4,27.9,28.1,26.9,28.8,32.6,27.2,28.2,26.2,23.5,34.5,24.5
1,1,-1,34.1,30.8,35.1,27.2,30.4,35.6,36.8,26.5,29.2,32.9,33.8,35.9,31.6,30.4,30.5
2,-1,1,44.5,38.4,44.2,45.7,63.9,49.5,42.3,45.3,54.7,43.5,46.5,46.8,44.8,41.5,30.6
3,1,1,51.9,55.9,58.9,57.4,58.3,60.2,60.8,56.4,52.3,61.2,57.3,62.5,58.5,53.5,52.4


In [3]:
factorial_eff = pd.DataFrame(growth_marigold_df, columns=['A','B','AB'])

In [4]:
# Add interaction effects
factorial_eff['AB'] = factorial_eff['A']*factorial_eff['B']

In [5]:
# Observations from three replicates
growth_y = np.array([[34.2, 26.2, 32.6, 33.4, 27.9, 28.1, 26.9, 28.8, 32.6, 27.2, 28.2, 26.2, 23.5, 34.5, 24.5],
                     [34.1, 30.8, 35.1, 27.2, 30.4, 35.6, 36.8, 26.5, 29.2, 32.9, 33.8, 35.9, 31.6, 30.4, 30.5],
                     [44.5, 38.4, 44.2, 45.7, 63.9, 49.5, 42.3, 45.3, 54.7, 43.5, 46.5, 46.8, 44.8, 41.5, 30.6],
                     [51.9, 55.9, 58.9, 57.4, 58.3, 60.2, 60.8, 56.4, 52.3, 61.2, 57.3, 62.5, 58.5, 53.5, 52.4]])

# Get a vector of total response
total_growth = np.c_[growth_y.sum(axis=1)]
factorial_eff, total_growth

(   A  B  AB
 0 -1 -1   1
 1  1 -1  -1
 2 -1  1  -1
 3  1  1   1,
 array([[434.8],
        [480.8],
        [682.2],
        [857.5]]))

In [6]:
k, n = np.log2(len(factorial_eff)), growth_y.shape[1]

contrast_eff = factorial_eff.iloc[:,0:].mul(total_growth).sum()
print('Contrast effects\n', contrast_eff.to_string(), sep='')

effects = (contrast_eff)/((2**(k-1))*n)
print('\nEffect estimates\n', effects.to_string(), sep='')

ss_eff = (contrast_eff**2)/((2**k)*n)
print('\nSS effects\n', ss_eff.to_string(), sep='')

Contrast effects
A     221.3
B     624.1
AB    129.3

Effect estimates
A      7.376667
B     20.803333
AB     4.310000

SS effects
A      816.228167
B     6491.680167
AB     278.641500


### SSE and MSE

In [7]:
SST = (sum(sum(growth_y**2)) - sum(total_growth)**2/growth_y.size)[0]
SSE = SST - sum(ss_eff)
MSE = SSE/(growth_y.size - len(ss_eff) - 1)
print("SSE = {:.2f}, MSE = {:.2f}".format(SSE, MSE))

SSE = 1229.27, MSE = 21.95


In [8]:
import statsmodels.api as sm

growth_model = sm.OLS(total_growth,factorial_eff).fit()
growth_model.summary2()

C:\Users\win10x64\anaconda\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Results: Ordinary least squares
================================================================================
Model:                  OLS              Adj. R-squared (uncentered): -2.719    
Dependent Variable:     y                AIC:                         68.7092   
Date:                   2021-11-23 16:59 BIC:                         66.8681   
No. Observations:       4                Log-Likelihood:              -31.355   
Df Model:               3                F-statistic:                 0.02517   
Df Residuals:           1                Prob (F-statistic):          0.992     
R-squared (uncentered): 0.070            Scale:                       1.5071e+06
-------------------------------------------------------------------------------------
         Coef.        Std.Err.        t         P>|t|         [0.025          0.975] 
-------------------------------------------------------------------------------------
A        55.3250      613.8250      0.0901      0.9428      -7744.0611      7854.7111
B       156.0250      613.8250      0.2542      0.8415      -7643.3611      7955.4111
AB       32.3250      613.8250      0.0527      0.9665      -7767.0611      7831.7111
--------------------------------------------------------------------------------
Omnibus:                   nan               Durbin-Watson:                0.000
Prob(Omnibus):             nan               Jarque-Bera (JB):             1.500
Skew:                      0.000             Prob(JB):                     0.472
Kurtosis:                  0.000             Condition No.:                1    
================================================================================

"""

In [9]:
SST - (sum(sum(growth_y**2))- sum(total_growth)**2/growth_y. size)[0]
SSE = SST - sum(ss_eff)
MSE = SSE/(growth_y.size- len(ss_eff)-1)
n=6
df_error= (2**2)*(n-1)
F_a = ss_eff[0] / (SSE/df_error)
F_b= ss_eff[1] / (SSE/df_error)
F_ab = ss_eff[2] / (SSE/df_error)
print("SSE ={:.2f}, MSE- {: .2f}". format(SSE, MSE))
print("F-A",F_a)
print("F-B",F_b)
print("F-AB",F_ab)
p_value_a = stats.f.sf(F_a,1,df_error)
p_value_b = stats.f.sf(F_b,  1, df_error)
p_value_ab = stats.f.sf(F_ab, 1, df_error)
print("P-value A",p_value_a)
print("P-value B",p_value_b)
print("P-value AB",p_value_ab)

0.0

SSE =1229.27, MSE-  21.95
F-A 13.279862661260376
F-B 105.61828735489765
F-AB 4.533439303913092
P-value A 0.0016139148924824635
P-value B 1.994153707145001e-09
P-value AB 0.04585300168761242
